### Не заработало. Зависает и ничего не делает.

https://github.com/ELS-RD/kernl/blob/main/tutorial/t5%20e2e.ipynb

In [1]:
import torch
from sage.spelling_correction import (
    RuM2M100ModelForSpellingCorrection,
    AvailableCorrectors,
)
from datetime import datetime

"""API to T5-based models for spelling correction.

To load a model:

    from corrector import AvailableCorrectors

    model = T5ModelForSpellingCorruption.from_pretrained(AvailableCorrectors.fred_large.value)
    ...
"""

import os
from typing import List, Optional, Union, Any
from tqdm import tqdm
from transformers import T5ForConditionalGeneration, AutoTokenizer

from sage.spelling_correction.corrector import Corrector


class T5ModelForSpellingCorruption(Corrector):
    """T5-based models."""

    def __init__(self, model_name_or_path: Union[str, os.PathLike]):
        self.model_name_or_path = model_name_or_path
        self.max_model_length = 512

    @classmethod
    def from_pretrained(cls, model_name_or_path: Union[str, os.PathLike]):
        engine = cls(model_name_or_path)
        engine.model = T5ForConditionalGeneration.from_pretrained(
            model_name_or_path,
        )
        engine.tokenizer = AutoTokenizer.from_pretrained(
            model_name_or_path, eos_token="</s>"
        )

        return engine

    def batch_correct(
        self,
        sentences: List[str],
        batch_size: int,
        prefix: Optional[str] = "",
    ) -> List[List[Any]]:
        """Correct multiple sentences"""

        batches = [
            sentences[i : i + batch_size] for i in range(0, len(sentences), batch_size)
        ]
        result = []
        # pb = tqdm(total=len(batches))
        # device = self.model.device
        for batch in batches:
            batch_prefix = [prefix + sentence for sentence in batch]
            batch_prefix = batch_prefix[0]
            encodings = self.tokenizer(
                batch_prefix,
                pad_to_multiple_of=8,
                padding=True,
                return_tensors="pt",
            ).to("cuda")
            with torch.inference_mode(), torch.autocast(
                dtype=torch.float32, cache_enabled=True, device_type="cuda"
            ):
                generated_tokens = self.model.generate(
                    **encodings,
                    max_length=encodings["input_ids"].shape[1] + 1,
                )
            ans = self.tokenizer.batch_decode(
                generated_tokens, skip_special_tokens=True
            )
            result.append(ans)
            # pb.update(1)
        return result


corrector = T5ModelForSpellingCorruption.from_pretrained(
    AvailableCorrectors.ent5_large.value
)

corrector.model.to(torch.device("cuda"))
None

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/user-name-goes-here/.local/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [2]:
import pandas as pd
from sage.evaluation.evaluate import evaluation

dataset_name_or_path = "t5_optimization/sage/data/example_data/jfleg"
src_file = open(os.path.join(dataset_name_or_path, "sources.txt"))
corr_file = open(os.path.join(dataset_name_or_path, "corrections.txt"))
sources = src_file.read().split("\n")
corrections = corr_file.read().split("\n")

size = 3
if size == -1:
    size = len(sources)
batch_size = 1
prefix = "grammar: "
answers = corrector.batch_correct(
    sources[:size],
    batch_size,
    prefix,
)
answers = sum(answers, [])
metrics = evaluation(sources, corrections, answers)
print(metrics)
# answers

  0%|          | 3/1602 [00:00<00:09, 169.93it/s]

{'Precision': 80.0, 'Recall': 80.0, 'F1': 80.0}


In [ ]:
import torch._dynamo as torchdynamo
from kernl.model_optimization import optimize_model

optimize_model(corrector.model.encoder)
optimize_model(corrector.model.decoder)

In [ ]:
import pandas as pd
from sage.evaluation.evaluate import evaluation


dataset_name_or_path = "t5_optimization/sage/data/example_data/jfleg"
src_file = open(os.path.join(dataset_name_or_path, "sources.txt"))
corr_file = open(os.path.join(dataset_name_or_path, "corrections.txt"))
sources = src_file.read().split("\n")
corrections = corr_file.read().split("\n")

size = 10
if size == -1:
    size = len(sources)
batch_size = 1
prefix = "grammar: "
answers = corrector.batch_correct(
    sources[:size],
    batch_size,
    prefix,
)
answers = sum(answers, [])
metrics = evaluation(sources, corrections, answers)
print(metrics)

In [2]:
import torch._dynamo as torchdynamo
from kernl.model_optimization import optimize_model

torchdynamo.config.cache_size_limit = 512

In [3]:
optimize_model(corrector.model.encoder)
optimize_model(corrector.model.decoder)

In [5]:
# corrector.model.to(torch.float16)
# corrector.model.eval()
# None

In [6]:
# corrector.tokenizer

In [3]:
metrics

{'Precision': 91.67, 'Recall': 73.33, 'F1': 81.48}

In [ ]:
with torch.inference_mode(), torch.autocast(
    dtype=torch.float16, cache_enabled=True, device_type="cuda"
):

    start = datetime.now()

    metrics = corrector.evaluate(

        "t5_optimization/sage/data/example_data/jfleg",
        batch_size=1,
        prefix="grammar: ",
    )


    duration = datetime.now() - start
    print(duration)

    print(metrics)

выдает ошибку.

In [5]:
input_ids = corrector.tokenizer(
    "translate English to French: The house in the woods is wonderful, can we buy it ?",
    return_tensors="pt",
    pad_to_multiple_of=8,
    padding=True,
).to("cuda")

with torch.inference_mode(), torch.autocast(
    dtype=torch.float16, cache_enabled=True, device_type="cuda"
):
    # for _ in range(10):
    #     output = corrector.model.generate(
    #         inputs=input_ids["input_ids"],
    #         min_length=22,
    #         max_length=22,
    #     )
    output = corrector.model.generate(
        inputs=input_ids["input_ids"],
        # min_length=22,
        # max_length=22,
    )

### official version

In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import time
import torch._dynamo as torchdynamo
import torch
from kernl.model_optimization import optimize_model

# default cache size needs to be increased to store the many graphs with generative models
torchdynamo.config.cache_size_limit = 512

model_name = "t5-small"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model = model.eval().cuda()

tokenizer = AutoTokenizer.from_pretrained(model_name)

input_ids = tokenizer(
    "translate English to French: The house in the woods is wonderful, can we buy it ?",
    return_tensors="pt",
    pad_to_multiple_of=8,
    padding=True,
).to("cuda")

with torch.inference_mode(), torch.autocast(
    dtype=torch.float16, cache_enabled=True, device_type="cuda"
):
    for _ in range(10):
        output = model.generate(
            inputs=input_ids["input_ids"],
            min_length=22,
            max_length=22,
        )
    torch.cuda.synchronize()
    start = time.perf_counter()
    output = model.generate(
        inputs=input_ids["input_ids"],
        min_length=22,
        max_length=22,
    )
    torch.cuda.synchronize()
    latency_baseline = time.perf_counter() - start
    print(latency_baseline)
    print(
        tokenizer.decode(
            output[0], skip_special_tokens=True, clean_up_tokenization_spaces=True
        )
    )

optimize_model(model.encoder)
optimize_model(model.decoder)


# warmup (IRL, encoder and decoder should be warmed each on their own)
with torch.inference_mode(), torch.autocast(
    dtype=torch.float16, cache_enabled=True, device_type="cuda"
):
    start = time.perf_counter()
    model.generate(inputs=input_ids["input_ids"], min_length=22, max_length=22)
    print(time.perf_counter() - start)


with torch.inference_mode(), torch.autocast(
    dtype=torch.float16, cache_enabled=True, device_type="cuda"
):
    for _ in range(10):
        model.generate(
            inputs=input_ids["input_ids"],
            min_length=22,
            max_length=22,
        )
    torch.cuda.synchronize()
    start = time.perf_counter()
    output = model.generate(
        inputs=input_ids["input_ids"],
        min_length=22,
        max_length=22,
    )
    torch.cuda.synchronize()
    latency_optimized = time.perf_counter() - start
    print(latency_optimized)
    print(f"{latency_baseline/latency_optimized:.1f}x speedup")
    print(
        tokenizer.decode(
            output[0], skip_special_tokens=True, clean_up_tokenization_spaces=True
        )
    )

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
config.json: 100%|██████████| 1.21k/1.21k [00:00<00:00, 5.79MB/s]
model.safetensors: 100%|██████████| 242M/242M [00:01<00:00, 128MB/s]  
generation_config.json: 100%|██████████| 147/147 [00:00<00:00, 490kB/s]
tokenizer_config.json: 100%|██████████| 2.32k/2.32k [00:00<00:00, 12.0MB/s]
spiece.model: 100%|██████████| 792k/792k [00:00<00:00, 1.82MB/s]
tokenizer.json: 100%|██████████| 1.39M/1.39M [00:00<00:00, 12.3MB/s]


0.16454567096661776
La maison dans les bois est merveilleuse, pouvons-nous l'acheter? 
205.29581649205647
0.06526946800295264
2.5x speedup
La maison dans les bois est merveilleuse, pouvons-nous l'acheter? 


In [ ]:
with torch.inference_mode(), torch.autocast(
    dtype=torch.float16, cache_enabled=True, device_type="cuda"
):

    for _ in range(10):
        model.generate(

            inputs=input_ids["input_ids"],
            min_length=22,

            max_length=22,
        )